# Proyecto del módulo: _Procesamiento de datos con Python_
---
### Limpieza de datos de un dataset de hospedajes de Airbnb de NY del año 2019

> Curso de Data Análysis de BEDU  
> Equipo 21:  
  * Adrián Camacho
  * Alan Fuentes
  * David Hernández
  * Hiram Ugarte

## Tipos de variables

Importación de dependencias y del dataset

In [ ]:
pip install validators zipcodes keplergl > /dev/null

In [ ]:
import pandas as pd
import numpy as np 
import random
from sklearn.model_selection import train_test_split
import re
import nltk
import unicodedata
from textblob import TextBlob
import spacy
from datetime import datetime
from nltk.stem import SnowballStemmer
import cufflinks as cf
from plotly.offline import plot,iplot
import plotly.graph_objects as go
import plotly.express as px
from palettable.scientific.diverging import Berlin_20
from PIL import Image
import requests
import validators
import psutil
from nltk.corpus import stopwords
from nltk import FreqDist
import seaborn as sns
import matplotlib.pyplot as plt
from palettable.cartocolors.qualitative import Bold_6
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from scipy import stats
import itertools
from scipy.stats import chisquare
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [ ]:
url = 'https://github.com/DavidHC11/Bedu21/blob/main/proyecto_r/listings2019.csv.gz?raw=true'
df = pd.read_csv(url, compression='gzip')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (61,62,94,95) have mixed types.Specify dtype option on import or set low_memory=False.



### Funciones

In [ ]:
# Gráficos
def bar(df,col,title,top,x_title="",y_title=""):
    layout = go.Layout(font_family="Arial",
    font_color="black",title_text=title,title_font_size=27,xaxis= {"title": {"text": x_title,"font": {"family": 'Arial',"size": 18,
        "color": '#000000'}}},yaxis= {"title": {"text": y_title,"font": {"family": 'Arial',"size": 18,
        "color": '#000000'}}},title_font_family="Arial",title_font_color="#000000",template="ggplot2")
    g_aux=pd.DataFrame(df[col].value_counts().head(top)).reset_index().rename(columns={"index":"conteo"})
    fig=g_aux.iplot(kind='bar',x="conteo",y=col,title=title,asFigure=True,barmode="overlay",sortbars=True,color="#2F789C",layout=layout)
    fig.update_layout(width=600)
    fig.update_traces(marker_color='#2F789C')
    return fig
def box(df,col,title):
    layout = go.Layout(font_family="Arial",
    font_color="black",title_text=title,title_font_size=30,xaxis= {"title": {"font": {"family": 'Arial',"size": 18,
        "color": '#002e4d'}}},title_font_family="Arial",title_font_color="#004878",template="plotly_white")
    fig=df[[col]].iplot(kind='box',title=title,asFigure=True,theme="white",layout=layout,color="#2F789C")
    return fig

def histogram(df,col,bins,title):
    layout = go.Layout(font_family="Arial",
    font_color="black",title_text=title,title_font_size=30,xaxis= {"title": {"font": {"family": 'Arial',"size": 18,
        "color": '#002e4d'}}},title_font_family="Arial",title_font_color="#004878",template="plotly_white")
    fig=df[[col]].iplot(kind='histogram',x=col,bins=bins,title=title,asFigure=True,theme="white",layout=layout,color="#2F789C")
    fig.update_traces(opacity=0.90)
    return fig

def pie(df,col,title,x_title="",y_title=""):
    layout = go.Layout(template="plotly_white")
    colors=[ "#2F789C","#60D394","#AAF683","#FFD97D","#FF9B85","#00569c","#005ba3","#0061a9","#1567af","#226cb6","#2c72bc", "#0061a9","#4c79b7","#7492c6","#98acd4","#bbc7e2","#dde3f1","#ffffff"
]
    aux=pd.DataFrame(df[col].value_counts()).reset_index().rename(columns={"index":"conteo"})
    fig=aux.iplot(kind='pie',labels="conteo",values=col,title=title,asFigure=True,theme="white")
    
    fig.update_traces(textfont_size=10,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
    fig.update_traces(textposition='inside', textinfo='percent+label')
    fig.update_layout(font_family="Arial",
    font_color="black",title_text=title,title_font_size=30,title_font_family="Arial",title_font_color="#004878",template="plotly_white")
    return fig

#Funciones
def clean_text(text, pattern="[^a-zA-Z0-9]"):
    cleaned_text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore')
    cleaned_text = re.sub(pattern, " ", cleaned_text.decode("utf-8"), flags=re.UNICODE)
    cleaned_text = u' '.join(cleaned_text.lower().strip().split())
    return cleaned_text

def completitud(data):
    cp=pd.DataFrame(data.isnull().sum())
    cp.reset_index(inplace=True)
    cp=cp.rename(columns={"index":"columna",0:"total"})
    cp["completitud"]=(1-cp["total"]/data.shape[0])
    cp=cp.sort_values(by="completitud")
    cp=cp[cp['total']>0]
    cp.reset_index(drop=True,inplace=True)
    return cp

def clean(data,col):
    data=data[data[col]!='']
    data.reset_index(drop=True,inplace=True)
    lista_stopwords_en = stopwords.words("english")
    data[col]=data[col].map(lambda x:" ".join(list(filter(lambda x:x not in lista_stopwords_en,str(x).split()))).strip())
    data=data[data[col]!='']
    data.reset_index(drop=True,inplace=True)
    corpus=' '.join(data[col].values)
    fdist=nltk.FreqDist(corpus.split())
    list_hapaxes=fdist.hapaxes()
    data[col]=data[col].map(lambda text:' '.join([x for x in text.split(' ') if x not in list_hapaxes]))
    data=data[data[col]!='']
    data.reset_index(drop=True,inplace=True)
    return data

def OUTLIERS(df,cols):
    results=pd.DataFrame()
    data_iqr=df.copy()
    data_per=df.copy()
    total=[]
    total_per=[]
    indices_=[]

    for col in cols:
        #IQR
        Q1=df[col].quantile(0.25)
        Q3=df[col].quantile(0.75)
        IQR=Q3-Q1
        INF=Q1-1.5*(IQR)
        SUP=Q3+1.5*(IQR)
    
        
        n_outliers=df[(df[col] < INF) | (df[col] > SUP)].shape[0]
        total.append(n_outliers)
        indices_iqr=list(df[(df[col] < INF) | (df[col] > SUP)].index)
        
        #Percentiles
        INF_pe=np.percentile(df[col].dropna(),5)
    
        SUP_pe=np.percentile(df[col].dropna(),95)
        n_outliers_per=df[(df[col] < INF_pe) | (df[col] > SUP_pe)].shape[0]
        total_per.append(n_outliers_per)
        indices_per=list(df[(df[col] < INF_pe) | (df[col] > SUP_pe)].index)
        
        
        indices_.append(aux_outliers(indices_iqr,indices_per))
        
    results["features"]=cols
    results["n_outliers_IQR"]=total
    results["n_outliers_Percentil"]=total_per
    results["n_outliers_IQR_%"]=round((results["n_outliers_IQR"]/df.shape[0])*100,2)
    results["n_outliers_Percentil_%"]=round((results["n_outliers_Percentil"]/df.shape[0])*100,2)
    results["indices"]=indices_
    results["total_outliers"]=results["indices"].map(lambda x:len(x))
    results["%_outliers"]=results["indices"].map(lambda x:round(((len(x)/df.shape[0])*100),2))
    results=results[['features', 'n_outliers_IQR', 'n_outliers_Percentil',
                     'n_outliers_IQR_%', 'n_outliers_Percentil_%','total_outliers', '%_outliers','indices']]
    return results

def aux_outliers(a,b):
    a=set(a)
    b=set(b)
    
    a_=a.intersection(b)

    b_=b.intersection(a)


    outliers_index=list(set(list(a_)+list(b_)))
    return outliers_index

def tableoutliers(df,outliers):
    data=df.copy()
    lista=[df.shape[0]]
    lista_ca=["Inicial"]
    for col in list(outliers["features"].values):
        indices=list(itertools.chain(*list(outliers[outliers["features"]==col]["indices"].values))) 
        data=data[~data.index.isin(indices)]
        lista.append(data.shape[0])
        lista_ca.append(col)
    tabla_registros=pd.DataFrame()
    tabla_registros["v_feature"]=lista_ca
    tabla_registros["c_n_rows"]=lista
    
    return tabla_registros

def dropoutliers(df,outliers):
    indices = list(set(list(itertools.chain(*list(outliers["indices"].values)))))
    df_new=df[~df.index.isin(indices)].reset_index(drop=True)
    return df_new

def mode_miss(X_train,X_test,col,df):
    moda=X_train[col].value_counts().index[0]
    X_train[col]=X_train[col].fillna(moda)
    X_test[col]=X_test[col].fillna(moda)
    x_i=df[col].fillna(moda).value_counts()
    k=x_i.sum()
    p_i=df[col].dropna().value_counts(1)
    m_i=k*p_i
    chi=chisquare(f_obs=x_i,f_exp=m_i)
    p_val=chi.pvalue
    alpha=0.05
    if p_val<alpha:
        a=print("Rechazamos HO(La porporción de categorias es la misma que la general)")
    else:
        a=print("Aceptamos HO(La porporción de categorias es la misma que la general)")
    return a

def missings_digit(x):
    if sum([y.isdigit() for y in str(x)])>0:
        result=np.nan
    else:
        result=x
    return result  


def transform_white_backgroud(png_path):
    path_is_url = validators.url(png_path)
    path = requests.get(png_path, stream=True).raw if path_is_url else png_path
    picture = Image.open(path).convert("RGBA")
    image = Image.new("RGB", picture.size, "WHITE")
    image.paste(picture, (0, 0), picture)

    plt.imshow(image)
    
    mask = np.array(image)
    return mask  

def count_generator():
  for column in df.columns:
    yield df[column].value_counts(1)

def number_of_items(list_str):
  begin, end = list_str[0], list_str[-1]
  if begin == "(" or begin == "[":
    begin = "\\" + begin
  if end == ")" or end == "]":
    end = "\\" + end
  list_pattern = re.compile(f"{begin}.+{end}")
  has_items = bool(list_pattern.match(list_str))
  commas = list_str.count(",")
  items = commas + has_items
  return items

def verificar_zipcodes(zipcode):
  import zipcodes
  lista=[]
  for i in zipcode.unique():
      if str(i)!="nan":
          x=pd.DataFrame(zipcodes.matching(i))
          if x["state"][0]!="NY":
            lista.append((i,x["state"][0]))
  return lista

### Tipos de variables del dataset

In [ ]:
llaves=['id', #id de la propiedad anunciada
        'listing_url', #url de la propiedad
        'scrape_id', #id del corte de los datos
        'thumbnail_url', #Url de vista previa
        'medium_url', # Url de vista previa extendido
        'picture_url', #Url de imagenes
        'xl_picture_url', # Url de imagen extendida
        'host_id', # id del host
        'host_url', #url perfil del host
        'host_thumbnail_url', #url de vista previa perfil del host
        'host_picture_url', #url foto de host
        'license', #Licencia propiedad
       ]
#Continuas
c_feats=['host_listings_count', #Publicaciones realizadas por el host
         'host_total_listings_count', #Total de publicaciones realizadas por el host
         'latitude', #Latitud
         'longitude', #Longitud
         'bathrooms', #Numero de baños
         'square_feet', #Tamaño de la propiedad en pies
         'price', #Precio en pesos por noche
         'weekly_price', #Precio por semana
         'monthly_price', #Precio por mes
         'security_deposit', # Deposito de seguridad
         'cleaning_fee', #Cuota de limpieza
         'extra_people', #Precio por persona extra
         'minimum_nights_avg_ntm', # Promedio de noches minimas
         'maximum_nights_avg_ntm', #Promedio de noches maximas
]
#Discretas
v_feats=['host_response_rate', #Rate sobre respuesta del host
         'host_acceptance_rate', #Rate sobre tiempo de aprobacion del host
         'host_is_superhost', #Clasificacion de superhost
         'host_has_profile_pic', #Verificacion de foto de perfil host
         'host_identity_verified', #Host verificado
         'zipcode', #codigo postal
         'is_location_exact', #Verificacion localizacion exacta
         'accommodates', #Numero de huespedes
         'bedrooms', #Numero de habitaciones
         'beds', #Numero de camas
         'guests_included', #Huespedes incluidos por precio
         'minimum_nights', #Noches minimas
         'maximum_nights', #Noches maximas
         'minimum_minimum_nights', #Minimo de noches minimas
         'maximum_minimum_nights', #Maximo de noches minimas
         'minimum_maximum_nights', #Minimo de noches maximas
         'maximum_maximum_nights', #Maximo de noches maximas
         'has_availability', #Verificacion de disponibilidad
         'availability_30', #Dias disponibles en los siguientes 30 dias
         'availability_60',#Dias disponibles en los siguientes 60 dias
         'availability_90',#Dias disponibles en los siguientes 90 dias
         'availability_365',#Dias disponibles en los siguientes 365 dias
         'number_of_reviews', #Numero de criticas
         'number_of_reviews_ltm', #Numero de criticas en el ultimo mes 
         'reviews_per_month', #Criticas por mes    
         'review_scores_rating', #rate de criticas para la propiedad
         'review_scores_accuracy', #Rate sobre la veracidad de los datos
         'review_scores_cleanliness', #Rate sobre la limpieza
         'review_scores_checkin', #Rate sobre el checkin
         'review_scores_communication', #Rate sobre la comunicacion con el host
         'review_scores_location', #Rate sobre la ubicacion
         'review_scores_value', #Rate sobre la calidad
         'requires_license',#Requisito de licencia
         'instant_bookable', #Verificacion reserva instantanea
         'is_business_travel_ready', #Preparada para viajes de negocios
         'require_guest_profile_picture', #Solicita foto de huesped
         'require_guest_phone_verification', #Solicita telefono de huesped
         'calculated_host_listings_count', #Conteo de publicaciones del host
         'calculated_host_listings_count_entire_homes', #Conteo de publicaciones de casas enteras del host
         'calculated_host_listings_count_private_rooms', #Conteo de publicaciones de habitaciones privadas del host
         'calculated_host_listings_count_shared_rooms',#Conteo de publicaciones de habitaciones compartidas del host
         'neighbourhood_cleansed', #Alcaldia
         'calendar_updated', #Ultima actualizacion del calendario
         'property_type', #tipo de propiedad
         'cancellation_policy', #Politica de cancelacion
         'room_type', #Tipo de habitacion
         'bed_type' #tipo de cama
         ]
#Fechas
d_feats=['last_scraped', #Fecha en la que se obtuvieron los datos
         'host_since', #Fecha de registro del host
         'calendar_last_scraped', #Fecha de ultimo corte de informacion
         'first_review', #Fecha de primer review
         'last_review', #Fecha de ultimo review
         ] 
#Texto
t_feats=['name', #Titulo del anuncio
         'summary', #Resumen de la propiedad
         'space', # Resumen del espacio
         'description', #Descripcion de la propiedad
         'experiences_offered', #Experiencias ofrecidas por la propiedad
         'neighborhood_overview', #Resumen del vecindario
         'notes', #Notas del host
         'transit', #Accesibilidad de la ubicacion del inmueble
         'access', #Zonas disponibles en la casa
         'interaction', #Formas de comunicacion con host
         'house_rules', #Reglas de la propiedad  
         'host_name', #nombre del host
         'host_location', # Localización del host
         'host_about', #Descripcion del host
         'host_response_time', #Tiempo de respuesta del host
         'host_neighbourhood', #vecindario host
         'host_verifications', #documentos de verificacion del host
         'street', #calle de la propiedad
         'neighbourhood', #Colonia
         'neighbourhood_group_cleansed', #Grupo de Alcaldias
         'city', #Ciudad
         'state', #Estado
         'market', #Mercado de propiedades ubicacion
         'smart_location', #Ubicacion simplificada
         'country_code', #Codigo de pais
         'country', #Pais
         'amenities', #amenidades
         'jurisdiction_names', #claves localizacion
         ] 

c_feats_new=["c_"+x for x in c_feats]
v_feats_new=["v_"+x for x in v_feats]
d_feats_new=["d_"+x for x in d_feats]
t_feats_new=["t_"+x for x in t_feats]

df.rename(columns=dict(zip(d_feats,d_feats_new)),inplace=True)
df.rename(columns=dict(zip(v_feats,v_feats_new)),inplace=True)
df.rename(columns=dict(zip(t_feats,t_feats_new)),inplace=True)
df.rename(columns=dict(zip(c_feats,c_feats_new)),inplace=True)

In [ ]:
df.head()

,id,listing_url,scrape_id,d_last_scraped,t_name,t_summary,t_space,t_description,t_experiences_offered,t_neighborhood_overview,t_notes,t_transit,t_access,t_interaction,t_house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,t_host_name,d_host_since,t_host_location,t_host_about,t_host_response_time,v_host_response_rate,v_host_acceptance_rate,v_host_is_superhost,host_thumbnail_url,host_picture_url,t_host_neighbourhood,c_host_listings_count,c_host_total_listings_count,t_host_verifications,v_host_has_profile_pic,v_host_identity_verified,t_street,t_neighbourhood,v_neighbourhood_cleansed,...,c_extra_people,v_minimum_nights,v_maximum_nights,v_minimum_minimum_nights,v_maximum_minimum_nights,v_minimum_maximum_nights,v_maximum_maximum_nights,c_minimum_nights_avg_ntm,c_maximum_nights_avg_ntm,v_calendar_updated,v_has_availability,v_availability_30,v_availability_60,v_availability_90,v_availability_365,d_calendar_last_scraped,v_number_of_reviews,v_number_of_reviews_ltm,d_first_review,d_last_review,v_review_scores_rating,v_review_scores_accuracy,v_review_scores_cleanliness,v_review_scores_checkin,v_review_scores_communication,v_review_scores_location,v_review_scores_value,v_requires_license,license,t_jurisdiction_names,v_instant_bookable,v_is_business_travel_ready,v_cancellation_policy,v_require_guest_profile_picture,v_require_guest_phone_verification,v_calculated_host_listings_count,v_calculated_host_listings_count_entire_homes,v_calculated_host_listings_count_private_rooms,v_calculated_host_listings_count_shared_rooms,v_reviews_per_month
0,2595,https://www.airbnb.com/rooms/2595,20191204162729,2019-12-07,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...","- Spacious (500+ft²), immaculate and nicely fu...","Beautiful, spacious skylit studio in the heart...",none,Centrally located in the heart of Manhattan ju...,NaN,Apartment is located on 37th Street between 5t...,"Guests have full access to the kitchen, bathro...",I am a Sound Therapy Practitioner and Kundalin...,"Make yourself at home, respect the space and t...",NaN,NaN,https://a0.muscache.com/im/pictures/f0813a11-4...,NaN,2845,https://www.airbnb.com/users/show/2845,Jennifer,2008-09-09,"New York, New York, United States",A New Yorker since 2000! My passion is creatin...,within a day,85%,NaN,f,https://a0.muscache.com/im/users/2845/profile_...,https://a0.muscache.com/im/users/2845/profile_...,Midtown,6.0,6.0,"['email', 'phone', 'reviews', 'offline_governm...",t,t,"New York, NY, United States",Midtown,Midtown,...,$0.00,10,1125,10,10,1125,1125,10.0,1125.0,2 weeks ago,t,1,1,1,1,2019-12-07,48,7,2009-11-21,2019-11-04,94.0,9.0,9.0,10.0,10.0,10.0,9.0,f,NaN,NaN,f,f,strict_14_with_grace_period,t,t,1,1,0,0,0.39
1,3831,https://www.airbnb.com/rooms/3831,20191204162729,2019-12-07,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Greetings! We own a double-duplex brownst...,Urban retreat: enjoy 500 s.f. floor in 1899 br...,none,Just the right mix of urban center and local n...,NaN,B52 bus for a 10-minute ride to downtown Brook...,You will have exclusive use of and access to: ...,"We'll be around, but since you have the top fl...",Smoking - outside please; pets allowed but ple...,NaN,NaN,https://a0.muscache.com/im/pictures/e49999c2-9...,NaN,4869,https://www.airbnb.com/users/show/4869,LisaRoxanne,2008-12-07,"New York, New York, United States",Laid-back bi-coastal actor/professor/attorney.,within an hour,100%,NaN,f,https://a0.muscache.com/im/users/4869/profile_...,https://a0.muscache.com/im/users/4869/profile_...,Clinton Hill,1.0,1.0,"['email', 'phone', 'reviews', 'kba']",t,t,"Brooklyn, NY, United States",Brooklyn,Clinton Hill,...,$0.00,1,730,1,1,730,730,1.0,730.0,2 weeks ago,t,1,1,1,1,2019-12-07,295,75,2014-09-30,2019-11-22,90.0,9.0,9.0,10.0,9.0,10.0,9.0,f,NaN,NaN,f,f,moderate,f,f,1,1,0,0,4.67
2,5099,https://www.airbnb.com/rooms/5099,20191204162729,2019-12-06,Large Cozy 1 BR Apartment In Midtown East,My large 1 bedroom apartm

Se eliminan las variables que solo fungen como llaves o identificadores

In [ ]:
df = df.drop(llaves, axis=1)

## Completitud

Función de completitud para visualizar la completitud de las variables del dataset

In [ ]:
cp=completitud(df)
cp

,columna,total,completitud
0,v_host_acceptance_rate,50599,0.000000
1,t_jurisdiction_names,50583,0.000316
2,c_square_feet,50213,0.007629
3,c_monthly_price,45683,0.097156
4,c_weekly_price,44945,0.111741
5,t_notes,29976,0.407577
6,t_access,23543,0.534714
7,t_interaction,20526,0.594340
8,t_host_about,20290,0.599004
9,t_house_rules,19520,0.614222


In [ ]:
colors=dict(zip(list(cp["columna"].values),Berlin_20.hex_colors))
fig = px.bar(cp, x="columna", y="completitud",color="columna",
             color_discrete_sequence=Berlin_20.hex_colors, title="COMPLETITUD")
layout = go.Layout(yaxis=dict(tickformat=".2%"))
fig.update_layout(layout)
fig.show()

Eliminamos variables con completitud menor al 80%

In [ ]:
incompletas=cp['columna'][cp['completitud']<.80].tolist()
df=df.drop(incompletas, axis=1)

In [ ]:
# Permitimos mostrar todas las columnas del dataset
pd.options.display.max_columns = None

In [ ]:
df.shape

(50599, 67)

## Tratamiento de variables




Utilizamos un generador para inspeccionar y clasificar variables:

In [ ]:
counts = count_generator()

In [ ]:
next(counts)

2019-12-06    0.473527
2019-12-05    0.429297
2019-12-04    0.086523
2019-12-07    0.010652
Name: d_last_scraped, dtype: float64

In [ ]:
eliminar = ['t_experiences_offered',
            't_host_location',
            't_street',
            't_host_neighbourhood',
            't_city',
            't_state',
            't_smart_location',
            't_market',
            't_country_code',
            't_country',
            'v_bed_type',
            't_host_name',
            "d_last_scraped",
            "v_host_has_profile_pic",
            "v_is_location_exact",
            "c_bathrooms",
            "v_has_availability",
            "d_calendar_last_scraped",
            "v_requires_license",
            "v_is_business_travel_ready",
            "v_require_guest_profile_picture",
            "v_require_guest_phone_verification",
            "v_calculated_host_listings_count_shared_rooms"
            ]

conservar = ['t_name', 
             't_summary',
             't_description',
             't_host_verifications',
             't_neighbourhood',
             'v_neighbourhood_cleansed',
             't_neighbourhood_group_cleansed',
             'v_property_type',
             'v_room_type',
             't_amenities',
             'v_calendar_updated',
             'v_cancellation_policy',
             "d_host_since",
             "v_host_is_superhost",
             "c_host_listings_count",
             "c_host_total_listings_count",
             "v_host_identity_verified",
             "v_zipcode",
             "c_latitude",
             "c_longitude",
             "v_accommodates",
             "v_bedrooms",
             "v_beds",
             "c_price",
             "v_guests_included",
             "c_extra_people",
             'v_minimum_nights',
             'v_maximum_nights',
             'v_minimum_minimum_nights',
             'v_maximum_minimum_nights',
             'v_minimum_maximum_nights',
             'v_maximum_maximum_nights',
             'c_minimum_nights_avg_ntm',
             'c_maximum_nights_avg_ntm',
             'v_availability_30',
             'v_availability_60',
             'v_availability_90',
             'v_availability_365',
             "v_number_of_reviews",
             "v_number_of_reviews_ltm",
             "v_instant_bookable",
             "v_calculated_host_listings_count",
             "v_calculated_host_listings_count_entire_homes",
             "v_calculated_host_listings_count_private_rooms"
            ]

In [ ]:
df.drop(columns=eliminar, inplace=True)

In [ ]:
df.head()

,t_name,t_summary,t_description,d_host_since,v_host_is_superhost,c_host_listings_count,c_host_total_listings_count,t_host_verifications,v_host_identity_verified,t_neighbourhood,v_neighbourhood_cleansed,t_neighbourhood_group_cleansed,v_zipcode,c_latitude,c_longitude,v_property_type,v_room_type,v_accommodates,v_bedrooms,v_beds,t_amenities,c_price,v_guests_included,c_extra_people,v_minimum_nights,v_maximum_nights,v_minimum_minimum_nights,v_maximum_minimum_nights,v_minimum_maximum_nights,v_maximum_maximum_nights,c_minimum_nights_avg_ntm,c_maximum_nights_avg_ntm,v_calendar_updated,v_availability_30,v_availability_60,v_availability_90,v_availability_365,v_number_of_reviews,v_number_of_reviews_ltm,v_instant_bookable,v_cancellation_policy,v_calculated_host_listings_count,v_calculated_host_listings_count_entire_homes,v_calculated_host_listings_count_private_rooms
0,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...","Beautiful, spacious skylit studio in the heart...",2008-09-09,f,6.0,6.0,"['email', 'phone', 'reviews', 'offline_governm...",t,Midtown,Midtown,Manhattan,10018,40.75362,-73.98377,Apartment,Entire home/apt,1,0.0,1.0,"{TV,Wifi,""Air conditioning"",Kitchen,""Paid park...",$225.00,1,$0.00,10,1125,10,10,1125,1125,10.0,1125.0,2 weeks ago,1,1,1,1,48,7,f,strict_14_with_grace_period,1,1,0
1,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Urban retreat: enjoy 500 s.f. floor in 1899 br...,2008-12-07,f,1.0,1.0,"['email', 'phone', 'reviews', 'kba']",t,Brooklyn,Clinton Hill,Brooklyn,11238,40.68514,-73.95976,Guest suite,Entire home/apt,3,1.0,4.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",$89.00,1,$0.00,1,730,1,1,730,730,1.0,730.0,2 weeks ago,1,1,1,1,295,75,f,moderate,1,1,0
2,Large Cozy 1 BR Apartment In Midtown East,My large 1 bedroom apartment has a true New Yo...,My large 1 bedroom apartment has a true New Yo...,2009-02-02,f,1.0,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",f,Manhattan,Murray Hill,Manhattan,10016,40.74767,-73.97500,Apartment,Entire home/apt,2,1.0,1.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",$200.00,2,$100.00,3,21,3,3,21,21,3.0,21.0,2 weeks ago,19,19,19,19,78,8,f,moderate,1,1,0
3,BlissArtsSpace!,NaN,HELLO EVERYONE AND THANKS FOR VISITING BLISS A...,2009-02-03,f,1.0,1.0,"['email', 'phone', 'facebook', 'reviews', 'off...",f,Bedford-Stuyvesant,Bedford-Stuyvesant,Brooklyn,11216,40.68688,-73.95596,Apartment,Private room,2,1.0,1.0,"{Wifi,""Air conditioning"",Kitchen,""Pets live on...",$60.00,1,$30.00,29,730,29,29,730,730,29.0,730.0,23 months ago,30,60,90,365,49,0,f,strict_14_with_grace_period,1,0,1
4,Large Furnished Room Near B'way,Please don’t expect the luxury here just a bas...,Please don’t expect the luxury here just a bas...,2009-03-03,f,1.0,1.0,"['email', 'phone', 'facebook', 'reviews']",f,Manhattan,Hell's Kitchen,Manhattan,10019,40.76489,-73.98493,Apartment,Private room,2,1.0,1.0,"{TV,Wifi,""Air conditioning"",""Paid parking off ...",$79.00,1,$12.00,2,14,1,2,14,14,1.8,14.0,2 months ago,3,12,40,242,454,47,f,strict_14_with_grace_period,1,0,1


In [ ]:
df.shape

(50599, 44)

Observamos el tipo de dato de cada variable

In [ ]:
for i in conservar:
  print(str(i) + ' is a(n) ' + str(df[i].dtype))

t_name is a(n) object
t_summary is a(n) object
t_description is a(n) object
t_host_verifications is a(n) object
t_neighbourhood is a(n) object
v_neighbourhood_cleansed is a(n) object
t_neighbourhood_group_cleansed is a(n) object
v_property_type is a(n) object
v_room_type is a(n) object
t_amenities is a(n) object
v_calendar_updated is a(n) object
v_cancellation_policy is a(n) object
d_host_since is a(n) object
v_host_is_superhost is a(n) object
c_host_listings_count is a(n) float64
c_host_total_listings_count is a(n) float64
v_host_identity_verified is a(n) object
v_zipcode is a(n) object
c_latitude is a(n) float64
c_longitude is a(n) float64
v_accommodates is a(n) int64
v_bedrooms is a(n) float64
v_beds is a(n) float64
c_price is a(n) object
v_guests_included is a(n) int64
c_extra_people is a(n) object
v_minimum_nights is a(n) int64
v_maximum_nights is a(n) int64
v_minimum_minimum_nights is a(n) int64
v_maximum_minimum_nights is a(n) int64
v_minimum_maximum_nights is a(n) int64
v_maxim

### Variable `v_host_is_superhost`

Eliminación de variables con valores nulos

In [ ]:
df = df[~df["v_host_is_superhost"].isnull()]

### Variable `v_cancellation_policy`

Distribución de la variable

In [ ]:
df["v_cancellation_policy"].value_counts(1)

strict_14_with_grace_period    0.457830
flexible                       0.305840
moderate                       0.232752
super_strict_60                0.002138
strict                         0.000979
super_strict_30                0.000460
Name: v_cancellation_policy, dtype: float64

In [ ]:
df["v_cancellation_policy"] = df["v_cancellation_policy"].map(lambda policy: "strict" if "strict" in policy else policy)
df["v_cancellation_policy"].value_counts(1)

strict      0.461408
flexible    0.305840
moderate    0.232752
Name: v_cancellation_policy, dtype: float64

###Variable `v_property_type`


In [ ]:
#La cantidad de valores posibles para esta variable puede reducirse para proveer una distribución más uniforme.
props = df.groupby('v_property_type').t_name.count().sort_values(ascending=False).reset_index()
props

,v_property_type,t_name
0,Apartment,39313
1,House,4126
2,Townhouse,1672
3,Condominium,1616
4,Loft,1404
5,Guest suite,421
6,Boutique hotel,361
7,Serviced apartment,343
8,Hotel,229
9,Other,102


In [ ]:
#Se crea una serie de funciones lambda para integrar algunos de los valores con otros similares,
#l primera para valores con mínimas incidencias:
is_other = lambda x : 'Other' if int(props[props.v_property_type == x].t_name) < 100 else x
#Y las siguientes para integrar términos estándar con sus respectivos eufemismos. Los términos a 
is_a_house = lambda x : 'House' if x in ['Earth house', 'Tiny house', 'Casa particular (Cuba)', 'Guesthouse', 'Condominium'] else x #Para integrar a 'House' sus eufemismos.
is_a_villa = lambda x : 'Townhouse' if x in ['Bungallow', 'Cottage'] else x #Para integrar a 'Villa' sus eufemismos.
is_a_hotel = lambda x : 'Hotel' if x in ['Bed and breakfast', 'Boutique hotel', 'Aparthotel', 'Resort'] else x #Para integrar a 'Hotel' sus eufemismos.
is_a_single_room = lambda x : 'Single Room' if x in ['Loft', 'Guest suite'] else x
is_an_apt = lambda x : 'Apartment' if x == 'Serviced apartment' else x

In [ ]:
for i in [is_a_house, is_a_villa, is_a_hotel, is_an_apt, is_other, is_a_single_room]:
  df['v_property_type'] = df['v_property_type'].map(i)

In [ ]:
df.groupby('v_property_type').t_name.count().sort_values(ascending=False).reset_index()

,v_property_type,t_name
0,Apartment,39656
1,House,5850
2,Single Room,1825
3,Townhouse,1680
4,Hotel,723
5,Other,286


###Variable `v_calendar_updated`

In [ ]:
df.groupby('v_calendar_updated').t_name.count().reset_index()

,v_calendar_updated,t_name
0,1 week ago,364
1,10 months ago,461
2,11 months ago,492
3,12 months ago,525
4,13 months ago,342
...,...,...
89,95 months ago,1
90,a week ago,2730
91,never,80
92,today,3996


In [ ]:
was_this_week = lambda x : 'This week' if x.find('days') != -1 else x
was_this_month = lambda x : 'This month' if x.find('week') != -1 else x
was_in_the_last_six_months = lambda x : 'Last 6 months' if x.find('months ago') != -1 and 6 >= int(x.split(' months ago')[0]) >= 2 else x
was_this_year = lambda x : 'This year' if x.find('months ago') != -1 and 12 >= int(x.split(' months ago')[0]) >= 7 else x
was_last_year = lambda x : 'Last year' if x.find('months ago') != -1 and 24 >= int(x.split(' months ago')[0]) >= 13 else x
was_long_ago = lambda x : 'Long ago' if x.find('months ago') != -1 and int(x.split(' months ago')[0]) > 24 else x
capitalize = lambda x : x.capitalize() if x in ['never', 'yesterday', 'today'] else x

In [ ]:


for i in [was_this_week, was_this_month, was_in_the_last_six_months, was_this_year, was_last_year, was_long_ago, capitalize]:
  df['v_calendar_updated'] = df.v_calendar_updated.apply(i)
df.groupby('v_calendar_updated').t_name.count()

v_calendar_updated
Last 6 months     8602
Last year         4037
Long ago          7884
Never               80
This month       20254
This year         3202
Today             3996
Yesterday         1965
Name: t_name, dtype: int64

### Variable de verificaciones `t_host_verifications`


Obtenemos la longitud de la lista de los métodos de verificación usados.

In [ ]:
df["t_host_verifications"].head()

0    ['email', 'phone', 'reviews', 'offline_governm...
1                 ['email', 'phone', 'reviews', 'kba']
2    ['email', 'phone', 'reviews', 'jumio', 'govern...
3    ['email', 'phone', 'facebook', 'reviews', 'off...
4            ['email', 'phone', 'facebook', 'reviews']
Name: t_host_verifications, dtype: object

Contamos la cantidad de métodos de verificación

In [ ]:
df["c_host_verifications"] = df["t_host_verifications"].map(number_of_items)
df["c_host_verifications"].value_counts().sort_index()

0        35
1       819
2      4138
3      3672
4      7816
5     10186
6     10138
7      6265
8      4031
9      2282
10      518
11      118
12       18
Name: c_host_verifications, dtype: int64

In [ ]:
df.drop(columns=["t_host_verifications"], inplace=True)

### Variable de comodidades `t_amenities`


In [ ]:
df["t_amenities"].head()

0    {TV,Wifi,"Air conditioning",Kitchen,"Paid park...
1    {TV,"Cable TV",Internet,Wifi,"Air conditioning...
2    {TV,"Cable TV",Internet,Wifi,"Air conditioning...
3    {Wifi,"Air conditioning",Kitchen,"Pets live on...
4    {TV,Wifi,"Air conditioning","Paid parking off ...
Name: t_amenities, dtype: object

Contamos el número de comodidades

In [ ]:
df["c_amenities"] = df["t_amenities"].map(number_of_items)
df["c_amenities"].value_counts()

14    2499
15    2482
16    2370
13    2314
17    2300
      ... 
77       1
82       1
81       1
78       1
76       1
Name: c_amenities, Length: 76, dtype: int64

Eliminamos la variable base

In [ ]:
df.drop(columns=["t_amenities"], inplace=True)

### Variable de vecindario `t_neighbourhood`

In [ ]:
pd.set_option('max_rows',1000)
df = df[~df['t_neighbourhood'].isnull()]

In [ ]:
df.shape

(50025, 44)

In [ ]:
df.columns

Index(['t_name', 't_summary', 't_description', 'd_host_since',
       'v_host_is_superhost', 'c_host_listings_count',
       'c_host_total_listings_count', 'v_host_identity_verified',
       't_neighbourhood', 'v_neighbourhood_cleansed',
       't_neighbourhood_group_cleansed', 'v_zipcode', 'c_latitude',
       'c_longitude', 'v_property_type', 'v_room_type', 'v_accommodates',
       'v_bedrooms', 'v_beds', 'c_price', 'v_guests_included',
       'c_extra_people', 'v_minimum_nights', 'v_maximum_nights',
       'v_minimum_minimum_nights', 'v_maximum_minimum_nights',
       'v_minimum_maximum_nights', 'v_maximum_maximum_nights',
       'c_minimum_nights_avg_ntm', 'c_maximum_nights_avg_ntm',
       'v_calendar_updated', 'v_availability_30', 'v_availability_60',
       'v_availability_90', 'v_availability_365', 'v_number_of_reviews',
       'v_number_of_reviews_ltm', 'v_instant_bookable',
       'v_cancellation_policy', 'v_calculated_host_listings_count',
       'v_calculated_host_listings_

###Variable de habitaciones `v_bedrooms`

#### Eliminación de Nulos

In [ ]:
df = df[~df["v_bedrooms"].isnull()]

####Estandarización de Formato

In [ ]:
df['v_bedrooms'] = df['v_bedrooms'].apply(int)
df['v_bedrooms'].value_counts().sort_index()

0      4604
1     35776
2      6706
3      2129
4       517
5       139
6        52
7        17
8        11
9         4
10        5
11        1
12        1
14        1
21        1
Name: v_bedrooms, dtype: int64

###Variable de habitaciones `v_beds`

####Eliminación de Nulos

In [ ]:
df = df[~df["v_beds"].isnull()]

####Estandarización de Formato

In [ ]:
df['v_beds'] = df['v_beds'].apply(int)
df['v_beds'].value_counts().sort_index()

0      1465
1     31296
2     10590
3      3757
4      1619
5       555
6       277
7       115
8        60
9        35
10       17
11       15
12       17
13        7
14        4
15        1
16        3
17        2
18        2
21        2
22        1
26        1
40        1
Name: v_beds, dtype: int64

### Variable de antigüedad del host `d_host_since`


#### Eliminación de valores nulos

In [ ]:
df = df[~df["d_host_since"].isnull()]

#### Filtrado solo mes y año

In [ ]:
def trim_day_from_date(date_str):
  return "-".join(date_str.split("-")[0:2])
df["d_host_since"] = df["d_host_since"].map(trim_day_from_date)
df["d_host_since"].value_counts().head(5)

2014-07    775
2015-07    769
2015-12    764
2014-05    743
2015-11    734
Name: d_host_since, dtype: int64

### Variable de huéspedes permitidos `v_accommodates`

In [ ]:
df["v_accommodates"] = df["v_accommodates"].map(int)

### Variable de precio `c_price`

In [ ]:
df["c_price"] = df["c_price"].str.replace("$","")
df["c_price"] = df["c_price"].str.replace(",","")
df["c_price"] = pd.to_numeric(df["c_price"])

### Variable de precio por persona adicional `c_extra_people`

In [ ]:
df["c_extra_people"] = df["c_extra_people"].str.replace("$","")
df["c_extra_people"] = df["c_extra_people"].str.replace(",","")
df["c_extra_people"] = pd.to_numeric(df["c_extra_people"])

### Variable de título de la publicación `t_name`

Descargamos las stopwords cargadas en el modulo para limpiar la variable.

In [ ]:
 nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Realizamos una función para limpiar las variables de texto, pero demora mucho tiempo, retirar los comentarios considerando un tiempo aproximado de 1 hr para las tres variables

In [ ]:
#df=clean(df,'t_name')

### Variable de resumen de la propiedad `t_summary`

In [ ]:
#df=clean(df,'t_summary')

###Variable de descripción de la propiedad `t_description`

In [ ]:
#df=clean(df,'t_description')

### Limpieza de texto final

Limpieza general de las variables de texto, se pasan a minúsculas, se retiran caracteres especiales y saltos de línea

In [ ]:
text=['t_name','t_summary','t_description','t_neighbourhood',	'v_neighbourhood_cleansed',	't_neighbourhood_group_cleansed','v_property_type','v_room_type','v_cancellation_policy']

for t in text:
    df[f'{t}']=df[f'{t}'].map(lambda x:clean_text(str(x).lower(), pattern="[^a-zA-Z0-9]").replace("\n",""))

### Variable `v_zipcode`

In [ ]:
df["v_zipcode"]=df["v_zipcode"].str.replace("NY ","")
df["v_zipcode"]=df["v_zipcode"].str.strip()
cambio={"11249\n11249":"11249",'11103-3233':"11103",'11413-3220':"11413",'10002-2289':"10002",'11385-2308':"11385"}
df["v_zipcode"].replace(cambio,inplace=True)
df["v_zipcode"].replace("",np.nan, inplace=True)
df=df[~df["v_zipcode"].isnull()]

In [ ]:
y=verificar_zipcodes(df["v_zipcode"])
y

[('91766', 'CA'), ('07093', 'NJ')]

##Análisis Exploratorio

#### A.E. `v_host_is_superhost`

Notamos que solamente el 19.7% de los anfitriones tienen la distinción de superhost que los hace destacar en la plataforma

In [ ]:
pie(df,'v_host_is_superhost','Super host',x_title="",y_title="")

#### A.E. `d_host_since`

Generamos variables adicionales para anilzar los meses con mayores registros y la evolución de anfitriones en los años de antigüedad de la app

In [ ]:
df['d_host_since_year']=df['d_host_since'].map(lambda x:x.split('-')[0])
df['d_host_since_month']=df['d_host_since'].map(lambda x:x.split('-')[1])

In [ ]:
bar(df,'d_host_since_year','Año de registro de anfitrión',11,x_title="Año",y_title="Número de anfitriones")

In [ ]:
bar(df,'d_host_since_month','Mes de registro de anfitrión',12,x_title="Mes",y_title="Número de anfitriones")

Observamos que en los meses de mayo, junio y julio, existe un aumento considerable en los registros de anfitriones, posiblimente coincida con la temporada vacacional. Para los registros por año encontramos un crecimiento exponencial hasta el 2015 con un decrecimiento por un estancamiento o estabilidad de la aplicación para los años posteriores.

#### A.E. `t_neighbourhood`

La gráfica demuestra que los barrios de Brooklyn y Manhattan concentran el mayor número de propiedades registradas en Airbnb.

In [ ]:
bar(df,'t_neighbourhood','Vecindario',12,x_title="Vecindario",y_title="Número de anfitriones")

#### A.E. Precio por noche en Brooklyn

Nos interesa conocer más sobre el barrio con mayor número de propiedades registradas. Encontramos que los anfitriones registraron un precio máximo por noche de \$10,000 USD y un mínimo de \$0 USD. El promedio indica un precio de \$125 USD por noche. No colocamos un boxplot porque la muestra presenta outliers que retiraremos posteriormente.

In [ ]:
g_aux=df[df['t_neighbourhood']=='brooklyn']
g_aux['c_price'].describe()

#### A.E. Ubicación

Queremos observar la distribución de las propiedades, por lo que aprovechammos los datos de longitud y latitud para generar un mapa.

In [ ]:
df['c_latitude']=df['c_latitude'].map(float)
df['c_longitude']=df['c_longitude'].map(float)
from keplergl import KeplerGl
map_1 = KeplerGl()
df_mapa=df[['t_name','c_latitude','c_longitude']]
map_1.add_data(data=df_mapa, name='data')
map_1.save_to_html()

#### A.E. `v_property_type`

La distribución de los tipos de propiedad con los patrones que muestran otras variables, coinciden en un concepto de estancias cortas, para un número bajo de huespedes, con un promedio de dos personas. Por ello el tipo de propiedad predominante son los departamentos.

In [ ]:
pie(df,'v_property_type','Tipo de propiedad',x_title="",y_title="")

#### A.E. `v_accommodates`

Confirmamos la información anterior, encontramos que predominan propiedades para un número bajo de huespedes. Por lo que la ciudad respeta su ambiente fluido y con bastante actividad turística.

In [ ]:
bar(df,'v_accommodates','Número de huespedes',11,x_title="Huespedes",y_title="Número de propiedades")

#### A.E. `v_cancellation_policy`

La política de cancelación tiene una ligera preferencia por una postura estricta con los huespedes, pero mantiene una distribución equitativa con los otros conceptos de cancelación.

In [ ]:
pie(df,'v_cancellation_policy','Política de cancelación',x_title="",y_title="")

####  A.E. `c_amenities`

La mayoría de las propiedades ofrecen un conjunto de amenidades entre 13 y 30 para la estancia de los huespedes.

In [ ]:
bar(df,'c_amenities','Número de amenidades',75,x_title="Número de amenidades",y_title="Número de propiedades")

In [ ]:
url='https://raw.githubusercontent.com/DavidHC11/Bedu21/main/proyecto_python/clean.png'
mask=transform_white_backgroud(url)
image_colors=ImageColorGenerator(mask)
stopwords=set(STOPWORDS)
word_cloud = WordCloud(mask=mask, background_color='white', contour_width=1, contour_color='grey', max_words=200, min_font_size=5, stopwords=stopwords).generate(" ".join(df["t_description"]))

plt.figure(figsize=(10,8))
plt.imshow(word_cloud.recolor(color_func=image_colors), interpolation="bilinear")
#word_cloud.to_file("python.png") #guardar la imagen
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

## Eliminación de Outliers

In [ ]:
df.filter(regex="^c_").columns

In [ ]:
outls=OUTLIERS(df, df.filter(regex="^c_").columns)
outls

In [ ]:
df["c_extra_people"].value_counts().head()

Los anfitriones no comparten el valor de los precios por huesped adicional, por lo que no brinda información la variable. Así que decidimos eliminar la variable asociada.

In [ ]:
df.drop(columns=["c_extra_people"], inplace=True)

In [ ]:
outls=OUTLIERS(df, df.filter(regex="^c_").columns)
outls

In [ ]:
tableoutliers(df,outls)

In [ ]:
df_new = dropoutliers(df,outls)
df_new.shape

In [ ]:
completitud(df_new) # Verificamos que ya no hay valores nulos

## Ingeniería de variables

Generamos variables dummies, para poder utilizarlas en un posterior modelo, asignamos el tipo de dato correcto para las variables de la tabla.

In [ ]:
df_aux=df_new.copy()
enteros=['v_zipcode','d_host_since_year','d_host_since_month']
dummies=['v_property_type','v_room_type','v_calendar_updated','v_cancellation_policy']
factor=['v_host_is_superhost','v_host_identity_verified','v_instant_bookable']

for col in factor:
  df_new[col]=df_new[col].map(lambda x: int( x=='t'))

for col in enteros:
  df_new[col]=df_new[col].map(int)

for c in dummies:
    df_new[c]=df_new[c].map(lambda x: str(x).replace(' ','_'))
dummy=pd.get_dummies(df_new[dummies],drop_first= True)
df_new=pd.concat([df_new[[x for x in df_new if x not in dummies]],dummy],axis=1)

### Transformación de `t_neighbourhood`

In [ ]:
vecindarios=pd.DataFrame(df_new['t_neighbourhood'].value_counts())

In [ ]:
vecindarios['vecindarios']=vecindarios.index
vecindarios.reset_index(inplace=True)
vecindarios['index']=vecindarios.index
vecindarios=vecindarios.set_index('vecindarios')

In [ ]:
num_veci=vecindarios['index'].to_dict()
df_new['t_neighbourhood'].replace(num_veci,inplace=True)

### Nueva variable `reservaciones_30`

Obtenemos el número de reservaciones en los siguientes 30 días

In [ ]:
df_new['reservaciones_30']=30 - df_new.v_availability_30

## Reducción de dimensiones

### Relación de valor perdido

Se revisa la completitud de las variables, la tabla nos muestra las variables incompletas

In [ ]:
completitud(df_new)

,columna,total,completitud


### Filtro de baja varianza

Se revisan las variables con baja varianza, no nos brinda información relevante por lo que se retirarán

In [ ]:
df_new.filter(like="c_").describe(percentiles=np.arange(0.1,1.1,.1))

,c_host_listings_count,c_host_total_listings_count,c_latitude,c_longitude,c_price,c_minimum_nights_avg_ntm,c_maximum_nights_avg_ntm,c_host_verifications,c_amenities
count,40510.000000,40510.000000,40510.000000,40510.000000,40510.000000,40510.000000,40510.000000,40510.000000,40510.000000
mean,2.002296,2.002296,40.728351,-73.955443,121.928215,4.966305,609.821145,5.330338,20.349914
std,2.631605,2.631605,0.053547,0.031038,70.751948,7.167621,536.874910,1.932578,9.272792
min,0.000000,0.000000,40.579340,-74.039420,0.000000,1.000000,1.000000,0.000000,0.000000
10%,1.000000,1.000000,40.668419,-73.994760,49.000000,1.000000,10.000000,3.000000,10.000000
20%,1.000000,1.000000,40.683768,-73.985842,60.000000,1.000000,21.000000,4.000000,12.000000
30%,1.000000,1.000000,40.693990,-73.975790,75.000000,2.000000,30.000000,4.000000,14.000000
40%,1.000000,1.000000,40.709730,-73.961810,89.000000,2.000000,90.000000,5.000000,16.000000
50%,1.000000,1.000000,40.720960,-73.954550,100.000000,2.300000,1125.000000,5.000000,18.000000
60%,1.000000,1.000000,40.734330,-73.948460,125.000000,3.000000,1125.000000,6.000000,21.000000


No se elimina ninguna columna por este filtro

### Filtro de alta correlación

Analizamos las variables correlacionadas entre sí y conservamos las que nos aporten mayor información para nuestro objetivo

In [ ]:
corr=df_new[[x for x in df_new if x!="c_price"]].corr(method="spearman")
corr=abs(corr)

In [ ]:
for col in corr.columns:
    display(corr[[col]][corr[[col]]>.8].dropna())

,v_host_is_superhost
v_host_is_superhost,1.0


,c_host_listings_count
c_host_listings_count,1.0
c_host_total_listings_count,1.0


,c_host_total_listings_count
c_host_listings_count,1.0
c_host_total_listings_count,1.0


,v_host_identity_verified
v_host_identity_verified,1.0


,t_neighbourhood
t_neighbourhood,1.0


...

,v_calendar_updated_Today
v_calendar_updated_Today,1.0


,v_calendar_updated_Yesterday
v_calendar_updated_Yesterday,1.0


,v_cancellation_policy_moderate
v_cancellation_policy_moderate,1.0


,v_cancellation_policy_strict
v_cancellation_policy_strict,1.0


,reservaciones_30
v_availability_30,1.000000
v_availability_60,0.937093
v_availability_90,0.908065
reservaciones_30,1.000000


Se retiran las siguiente variables por correlación y no aportar información adicional al modelo

In [ ]:
cols=['v_minimum_minimum_nights','v_maximum_minimum_nights','c_minimum_nights_avg_ntm',
      'v_minimum_maximum_nights', 'v_maximum_maximum_nights',	'c_maximum_nights_avg_ntm',
      'v_availability_60','v_availability_90','v_availability_365','v_number_of_reviews_ltm','c_host_total_listings_count',
      'v_calculated_host_listings_count_entire_homes','v_calculated_host_listings_count_private_rooms','v_availability_30']

df_new=df_new.drop(columns=cols)

### Correlación con la variable objetivo

In [ ]:
corr=df_new.corr(method="spearman")
corr=abs(corr)
corr[["c_price"]].sort_values(by = 'c_price',ascending = False).style.background_gradient()

,c_price
c_price,1.000000
v_room_type_private_room,0.625004
v_accommodates,0.537710
c_longitude,0.406168
v_beds,0.349124
v_zipcode,0.322151
v_guests_included,0.297656
v_bedrooms,0.226779
v_calculated_host_listings_count,0.198320
v_room_type_shared_room,0.182852


Se eliminan las variables con correlacion menor a 10%

In [ ]:
corr=corr[["c_price"]].sort_values(by = 'c_price',ascending = False)
cols_drop_low_corr=list(corr[corr["c_price"]<.1].index)
df_new=df_new.drop(columns=cols_drop_low_corr)

### Multicolinealidad

In [ ]:
texto=['t_name',	't_summary',	't_description',	'd_host_since','v_neighbourhood_cleansed',	't_neighbourhood_group_cleansed']
df_final=df_new.drop(texto,axis=1)

In [ ]:
def calc_vif(X):
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return(vif)

In [ ]:
for col in df_final.columns:
    df_final[col]=df_final[col].astype(float)
    
X=df_final[[x for x in df_final if x!="c_price"]]

In [ ]:
df_aux2=calc_vif(X)

In [ ]:
df_aux2[df_aux2["VIF"]>5]

,variables,VIF
1,v_zipcode,4.885911e+02
2,c_latitude,7.398642e+05
3,c_longitude,1.010013e+06
4,v_accommodates,1.225779e+01
5,v_bedrooms,6.978483e+00
6,v_beds,7.689223e+00
9,c_amenities,6.361739e+00
10,d_host_since_year,6.831747e+05


Nota: No se retiran las variables con VIF mayor a 5, pero se considera que un coeficiente de regresión para estas variables no se estima adecuadamente.

### Análisis de Componentes principales

Separamos el conjunto de datos en una tabla de entrenamiento y de prueba

In [ ]:
X_train,X_test=train_test_split(df_final,test_size=.2,random_state=413)
df_train=X_train[[x for x in X_train if x!="c_price"]]
df_test=X_train[[x for x in X_train if x!="c_price"]]
y_train=X_train[["c_price"]]
y_test=X_test[["c_price"]]

Realizamos un escalamiento de los datos, debido a que el análsis de componentes principales es sensible a las influencias de la distribución de los datos

In [ ]:
scaler = StandardScaler()
scaler.fit(df_train)
df_train_sc=pd.DataFrame(scaler.transform(df_train))
df_test_sc=pd.DataFrame(scaler.transform(df_train))

Realizamos una prueba con 8 componentes para conocer la varianza descrita por la combinación de las variables

In [ ]:
pca = PCA(n_components=8)
pca.fit(df_train_sc)

PCA(copy=True, iterated_power='auto', n_components=8, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [ ]:
X_train_pca=pd.DataFrame(pca.transform(df_train_sc))
X_test_pca=pd.DataFrame(pca.transform(df_test_sc))

Obtenemos los porcentajes de varianza descrita por los componentes

In [ ]:
pca.explained_variance_ratio_

array([0.22830389, 0.14788269, 0.13879055, 0.09451446, 0.08317621,
       0.07251092, 0.06521942, 0.05385333])

Con 7 componentes podemos representar un 83% de la información de nuestros datos

In [ ]:
varianza=list(pca.explained_variance_ratio_)
varianza=list(map(lambda x:sum(varianza[:x]),range(1,len(varianza)+1)))
cp=pd.DataFrame()
cp["n_componente"]=range(1,9)
cp["varianza"]=varianza
fig = go.Figure(data=go.Scatter(x=cp["n_componente"], y=cp["varianza"]))
fig.show()

# ¡Muchas gracias por su atención!